In [28]:
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
# https://www.tensorflow.org/decision_forests/tutorials/predict_colab
# https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab
# https://www.tensorflow.org/decision_forests/tutorials/dtreeviz_colab
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np

import math
import os

import tensorflow as tf
print(tf.__version__)

2.12.0


In [29]:
# ! pip install --upgrade tensorflow-decision-forests
import tensorflow_decision_forests as tfdf
print(tfdf.__version__)

1.3.0


# Variable to Configuration

In [30]:
projectId='pongthorn'
dataset_id='SMartML'

train_name='train2_incident'
test_name='test2_incident'

train_table_id=f"{projectId}.{dataset_id}.{train_name}"
test_tabel_id=f"{projectId}.{dataset_id}.{test_name}"
print(f"train-ds = {train_table_id}")
print(f"test-ds = {test_tabel_id}")

train-ds = pongthorn.SMartML.train2_incident
test-ds = pongthorn.SMartML.test2_incident


In [31]:
metric="accuracy"
model_gs_path="gs://demo-tuned-tf-incident-pongthorn/model_df_tf"

In [32]:
unusedCols=['id','severity_id','severity_name','label_binary_severity','open_to_close_hour','response_to_resolved_hour']
cateCols=['sla','product_type','brand','service_type','incident_type','range_open_to_close_hour','range_response_to_resolved_hour']
numbericCols=[]

# unusedCols=['id','severity_id','severity_name','label_binary_severity','range_open_to_close_hour','range_response_to_resolved_hour']
# cateCols=['sla','product_type','brand','service_type','incident_type']
# numbericCols=['open_to_close_hour','response_to_resolved_hour']


labelCol='label_multi_severity'
# labelCol='label_binary_severity'

# Load & Prepare Data

In [33]:
def load_ml_data(data_path):
 df=pd.read_csv(data_path)
 df =df.drop(columns=unusedCols)
 
 return df

def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 df =df.drop(columns=unusedCols)
 df[labelCol]=df[labelCol].astype('int64') 
 df=df[[labelCol]+cateCols+numbericCols]   
  
 return df

In [34]:
client = bigquery.Client(project=projectId)

train=load_data_bq(f"SELECT * FROM {train_table_id}")
test=load_data_bq(f"SELECT * FROM {test_tabel_id}")

print(train.info())

print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2104 entries, 0 to 2103
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   label_multi_severity             2104 non-null   int64 
 1   sla                              2104 non-null   object
 2   product_type                     2104 non-null   object
 3   brand                            2104 non-null   object
 4   service_type                     2104 non-null   object
 5   incident_type                    2104 non-null   object
 6   range_open_to_close_hour         2104 non-null   object
 7   range_response_to_resolved_hour  2104 non-null   object
dtypes: int64(1), object(7)
memory usage: 131.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 

In [35]:
train.tail(5)

,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
2099,2,24x7 4Hrs Response Time,Software,Trend Micro,Incident,General Incident,worst,worst
2100,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,General Incident,best,best
2101,2,24x7 4Hrs Response Time,Security,Trend Micro,Incident,Software,good,good
2102,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,Configuration Change,good,good
2103,2,24x7 6Hrs Resolution Time,Software,Trend Micro,Incident,General Incident,moderate,moderate


In [36]:
test.tail()

,label_multi_severity,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
522,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,best,best
523,1,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,good,good
524,0,24x7 4Hrs Response Time,Firewall,Palo Alto,Request,Configuration Change,bad,bad
525,1,24x7 4Hrs Response Time,Software,Trend Micro,Incident,Software,best,best
526,0,24x7 4Hrs Response Time,Software,Trend Micro,Request,General Incident,moderate,good


# Build Model

In [37]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label=labelCol)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label=labelCol)

In [38]:
# Specify the model.
#model = tfdf.keras.RandomForestModel()
model=tfdf.keras.GradientBoostedTreesModel()
model.fit(x=train_ds)

Use /var/tmp/tmpq6ybr5ac as temporary training directory
Reading training dataset...


[WARNING 23-06-27 15:42:57.0054 UTC gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-06-27 15:42:57.0064 UTC gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-06-27 15:42:57.0069 UTC gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-06-27 15:42:57.025160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [2104]
	 [[{{node Placeholder/_5}}]]


Training dataset read in 0:00:00.224452. Found 2104 examples.
Training model...
Model trained in 0:00:00.146388
Compiling model...
Model compiled.


[INFO 23-06-27 15:42:57.3748 UTC kernel.cc:1242] Loading model from path /var/tmp/tmpq6ybr5ac/model/ with prefix 988efd7dec634688
[INFO 23-06-27 15:42:57.3881 UTC decision_forest.cc:660] Model loaded with 76 root(s), 3708 node(s), and 7 input feature(s).
[INFO 23-06-27 15:42:57.3889 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-06-27 15:42:57.3895 UTC kernel.cc:1074] Use fast generic engine
2023-06-27 15:42:57.406651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype string and shape [2104]
	 [[{{node Placeholder/_3}}]]


In [39]:
model.compile(metrics=[metric])
evaluation = model.evaluate(test_ds, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 0s 184ms/step - loss: 0.0000e+00 - accuracy: 0.7324
loss: 0.0000
accuracy: 0.7324


2023-06-27 15:42:58.941321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [527]
	 [[{{node Placeholder/_0}}]]


# Save Model

In [40]:
model.save(model_gs_path)

INFO:tensorflow:Assets written to: gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets


INFO:tensorflow:Assets written to: gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets


# Load Model

In [41]:
abc_model = tf.keras.models.load_model(model_gs_path)  

[INFO 23-06-27 15:43:12.6801 UTC kernel.cc:1242] Loading model from path gs://demo-tuned-tf-incident-pongthorn/model_df_tf/assets/ with prefix 988efd7dec634688
[INFO 23-06-27 15:43:13.1018 UTC decision_forest.cc:660] Model loaded with 76 root(s), 3708 node(s), and 7 input feature(s).
[INFO 23-06-27 15:43:13.1027 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-06-27 15:43:13.1032 UTC kernel.cc:1074] Use fast generic engine


# Make Prediction

In [52]:
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
unseen=query_result.to_dataframe()
unusedCols_unseen=['id','severity_name','open_to_close_hour','response_to_resolved_hour']
# unusedCols_unseen=['id','severity_id','severity_name'','range_open_to_close_hour','range_response_to_resolved_hour']
unseen =unseen.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   severity_id                      25 non-null     Int64 
 1   sla                              25 non-null     object
 2   product_type                     25 non-null     object
 3   brand                            25 non-null     object
 4   service_type                     25 non-null     object
 5   incident_type                    25 non-null     object
 6   range_open_to_close_hour         25 non-null     object
 7   range_response_to_resolved_hour  25 non-null     object
dtypes: Int64(1), object(7)
memory usage: 1.7+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour,range_response_to_resolved_hour
0,2,24x7 4Hrs Response Time,Server,DELL,Incident,Network Adapter Failure,good,best
1,2,24x7 4Hrs Response Time,Storage,NetApp,Incident,Controller/Node Failure,good,best
2,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Controller/Node Failure,good,best
3,2,24x7 4Hrs Response Time,Storage,NetApp,Incident,Controller/Node Failure,good,best
4,2,24x7 4Hrs Response Time,Storage,NetApp,Incident,Controller/Node Failure,best,best
5,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,best,best
6,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,moderate,best
7,3,24x7 4Hrs Response Time,Software,Trend Micro,Incident,General Incident,good,good
8,3,24x7 4Hrs Response Time,Software,Trend Micro,Incident,General Incident,best,best
9,4,24x7 4Hrs Response Time,Software,Red Hat,Request,General Incident,good,good


In [53]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
unseen_ds

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_response_to_resolved_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>

In [54]:
predResultList=abc_model.predict(unseen_ds)

1/1 [==============================] - 0s 19ms/step


2023-06-27 15:50:16.740782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [25]
	 [[{{node Placeholder/_5}}]]


In [55]:
map_severity_to_class={0:4,1: 3, 2: 2, 3: 1}
for predResult in predResultList:
    _class=tf.argmax(predResult,-1).numpy()
    print(f"{predResult} : {_class} as severity#{map_severity_to_class[_class]}")   

[0.09771261 0.32907462 0.5240794  0.04913337] : 2 as severity#2
[0.05273436 0.17789167 0.68611556 0.08325837] : 2 as severity#2
[0.05273436 0.17789167 0.68611556 0.08325837] : 2 as severity#2
[0.05273436 0.17789167 0.68611556 0.08325837] : 2 as severity#2
[0.07804295 0.06752458 0.6410539  0.21337855] : 2 as severity#2
[0.04452145 0.47449166 0.43188232 0.04910461] : 1 as severity#3
[0.04693042 0.59754026 0.30376777 0.05176157] : 1 as severity#3
[0.04042179 0.6957068  0.22253098 0.04134038] : 1 as severity#3
[0.0381404  0.6564414  0.26641098 0.03900715] : 1 as severity#3
[0.84745497 0.11427605 0.01919164 0.01907746] : 0 as severity#4
[0.8456705  0.11403542 0.02125685 0.01903729] : 0 as severity#4
[0.03640204 0.3033496  0.6246186  0.03562966] : 2 as severity#2
[0.04643405 0.14778648 0.42696366 0.37881583] : 2 as severity#2
[0.04643405 0.14778648 0.42696366 0.37881583] : 2 as severity#2
[0.04643405 0.14778648 0.42696366 0.37881583] : 2 as severity#2
[0.0402318  0.09903562 0.7536914  0.1070